<a href="https://colab.research.google.com/github/dgadela/ML_HandsOn/blob/master/DigitRecogKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

#load Data
train=pd.read_csv("train_digit.csv")
test=pd.read_csv("test_digit.csv")
y_train=train['label']
x_train=train.drop(labels=['label'],axis=1)
y_train.value_counts()

#Normalisation
x_train=x_train/255.0
test=test/255.0

#reshape
x_train=x_train.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

#label encoding
y_train=to_categorical(y_train,num_classes=10)

#split training and testing dataset
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=2)

#define the model
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='same',activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10,activation="softmax"))

# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

#fit the model
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, 
validation_data = (x_val, y_val), verbose = 2)

results=model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
print(results)

367/367 - 345s - loss: 0.2536 - accuracy: 0.9213 - val_loss: 0.0713 - val_accuracy: 0.9785
0        2
1        0
2        9
3        9
4        3
        ..
27995    9
27996    7
27997    3
27998    9
27999    2
Name: Label, Length: 28000, dtype: int64
